# Objec-Oriented Design for Implementation

Since the linear regression model is the basic model in machine learning, it uses many of the same components that other models use. Therefore, before diving into the implementation details it is worth designing some of the APIs that we will use in the implementation.

Inspired by libraries like `PyTorch`, at the high level we wish to have three classes: (*i*) `Module` contains models, losses and optimization methods; (*ii*) `DataModule` provides data loaders for training and validation; (*iii*) both classes are combined using the `Trainer` class, which allows us to train models on a variety of hardware platforms.

In [ ]:
import time
import numpy as np
import torch
from torch import nn

## Utilities 
To make this notebook more readable, we will to define some utilities. The first one allows us to register a function as a method in a class after the class has been created. In fact, we can do so even after we have created instances of the class.

In [ ]:
def add_to_class(Class):
"""Register funcions as methods in created class."""
def wrapper(obj):
    setattr(Class, obj.__name__, obj)
return wrapper

Let's see how it works:

In [ ]:
class A:
    def __init__(self):
        self.b = 1

a = A()

In [ ]:
@add_to_class(A)
def do(self):
    print('Class attribute "b" is, self.b')

a.do()

The second one is a ultility class that saves all arguments in a class's `__init__` method as class attributes. This allows us to extend constructor call signatures without additional code.

In [ ]:
class HyperParemeters: #@save
    """The base class of hyperparameters."""
    def save_hyperparameters(self, ignore=[]):
        raise NotImplemented